In [1]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from pre_process import PreProcess

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd

data = pd.read_csv('./spam.csv')

In [3]:
data["Message"] = PreProcess.preProcess(data["Message"])

In [4]:
for i in range(0, data["Message"].size):
  data["Message"][i] = PreProcess.standardize(data["Message"][i])

In [5]:
data["Label"] = 0
for i in range(0, data["Category"].size):
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14764\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14764\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14764\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [6]:
X = data["Message"]
Y = data["Label"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [94]:
print(type(X_train))

<class 'pandas.core.series.Series'>


In [8]:
dim = 150
kernel_size = 3
max_length = 40

In [9]:
from gensim.models import Word2Vec
sent = list(X_train)
sentences = [row.split() for row in sent]
print(sentences)
w2v_model = Word2Vec(sentences, vector_size=dim, window=5, min_count=1, workers=4)
w2v_model.save("word2vec2.model")

[['good', 'movie', 'ok', 'leave', 'hourish'], ['free', 'give', 'otherwise', 'nalla', 'adi', 'entey', 'nattil', 'kittum'], ['emigrate', 'something', 'ok', 'maybe', 'bite', 'hopeful'], ['get', 'home', 'babe', 'still', 'awake'], ['kay', 'since', 'already'], ['workin', 'get', 'job'], ['say', 'matter', 'mind', 'say', 'matter'], ['oh', 'yeah', 'diet', 'fly', 'window'], ['sorry', 'get', 'thing', 'may', 'pub', 'late'], ['ill', 'call', 'even', 'ill', 'idea'], ['dear', 'reache', 'room'], ['get', 'look', 'scrumptious', 'daddy', 'want', 'eat', 'night', 'long'], ['badrith', 'chennai', 'surely', 'pick', 'u', 'competition'], ['da', 'car', 'park'], ['okay', 'soon', 'best'], ['yar', 'else', 'thk', 'sort', 'funny', 'thing'], ['yup', 'izzit', 'still', 'rain', 'heavily', 'co', 'e', 'mrt', 'c', 'outside'], ['way', 'ur', 'home'], ['ok', 'pop', 'ask', 'bout', 'something', 'say', 'around', 'tonght', 'wen', 'girl', 'come'], ['talk', 'g', 'x'], ['cool', 'tyler', 'take', 'gonna', 'buy', 'drop', 'place', 'late', 

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [11]:
from joblib import dump
dump(tokenizer, 'cnn_tokenizer')

['cnn_tokenizer']

In [12]:
x_data = tokenizer.texts_to_sequences(X_train)
x_data = pad_sequences(x_data, maxlen=max_length)
print(x_data[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0   12  352    9   66 2819]


In [13]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, dim))

for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [14]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, dim, weights=[embedding_matrix],
                    input_length=max_length))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='softmax'))  # Adjust NUM_CLASSES based on your task

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
from keras.layers import Flatten
from keras.layers import Dropout, Activation

model = Sequential()
model.add(Embedding(len(word_index) + 1, dim, weights=[embedding_matrix], input_length=max_length))
model.add(Dropout(0.4))
model.add(Conv1D(600, kernel_size, padding='valid', activation='relu', strides=1))
model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(600))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(x_data, Y_train, epochs=8, batch_size=128, validation_split=0.1)

Epoch 1/8


32/32 [==============================] - 6s 136ms/step - loss: 0.3462 - accuracy: 0.8606 - val_loss: 0.3181 - val_accuracy: 0.8632
Epoch 2/8
32/32 [==============================] - 4s 126ms/step - loss: 0.2737 - accuracy: 0.8743 - val_loss: 0.2603 - val_accuracy: 0.8924
Epoch 3/8
32/32 [==============================] - 4s 132ms/step - loss: 0.2128 - accuracy: 0.9112 - val_loss: 0.2131 - val_accuracy: 0.9462
Epoch 4/8
32/32 [==============================] - 4s 130ms/step - loss: 0.1071 - accuracy: 0.9678 - val_loss: 0.1320 - val_accuracy: 0.9619
Epoch 5/8
32/32 [==============================] - 4s 131ms/step - loss: 0.0484 - accuracy: 0.9838 - val_loss: 0.1116 - val_accuracy: 0.9843
Epoch 6/8
32/32 [==============================] - 4s 128ms/step - loss: 0.0187 - accuracy: 0.9925 - val_loss: 0.1470 - val_accuracy: 0.9798
Epoch 7/8
32/32 [==============================] - 4s 130ms/step - loss: 0.0104 - accuracy: 0.9958 - val_loss: 0.1711 - val_accuracy: 0.9821
Epoch 8/8
3

In [17]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 150)           860550    
                                                                 
 dropout (Dropout)           (None, 40, 150)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 600)           270600    
                                                                 
 conv1d_2 (Conv1D)           (None, 36, 300)           540300    
                                                                 
 conv1d_3 (Conv1D)           (None, 34, 150)           135150    
                                                                 
 conv1d_4 (Conv1D)           (None, 32, 75)            33825     
                                                                 
 flatten (Flatten)           (None, 2400)             

In [18]:
from sklearn.metrics import classification_report

test_padded = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_padded, maxlen=max_length)
prediction = model.predict(test_padded)
pred_labels = []
for i in prediction:
  if i >= 0.5:
    pred_labels.append(1)
  else:
    pred_labels.append(0)
print(classification_report(Y_test,pred_labels, digits=5))

35/35 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0    0.98045   0.99791   0.98910       955
           1    0.98601   0.88125   0.93069       160

    accuracy                        0.98117      1115
   macro avg    0.98323   0.93958   0.95990      1115
weighted avg    0.98125   0.98117   0.98072      1115



In [19]:
from joblib import dump
dump(model, 'model_cnn')

['model_cnn']

In [117]:
text = "fpa notice : ebay misrepresentation of identity - user suspension - section 9 -  dear ebay member ,  in an effort to protect your ebay  account security , we have suspended your account until such time that it can  be safely restored to you . we have taken this action because your account  may have been compromised . although we cannot disclose our investigative  procedures that led to this conclusion , please know that we took this action  in order to maintain the safety of your account . for instructions on  getting your account reinstated , please click the button bellow :  thank you for your patience and  cooperation . regards ,  safeharbor departmentebay  inc . "
email = pd.Series(data=[text], index=[0])
email = PreProcess.preProcess(email)
email[0] = PreProcess.standardize(email[0])
print(type(email))
email_padded = tokenizer.texts_to_sequences(email)
print(email_padded)
email_padded = pad_sequences(email_padded, maxlen=max_length)
prediction = model.predict(email_padded)
print(prediction[0])

if prediction[0] >= 0.5:
  print("spam")
else:
  print("ham")

<class 'pandas.core.series.Series'>
[[920, 4031, 761, 2555, 51, 4031, 832, 2931, 4031, 228, 228, 15, 4603, 25, 1424, 228, 174, 5034, 2144, 2589, 1232, 5657, 52, 11, 25, 1424, 363, 1780, 4884, 228, 1706, 2, 228, 52, 1441, 2855, 74, 757, 925]]
1/1 [==============================] - 0s 27ms/step
[0.00045066]
ham
